In [25]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint



### EJEMPLO


In [26]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year"


In [27]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [28]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
pprint(data)

{'data': {'attributes': {'description': 'Balance eléctrico: asignación de '
                                        'unidades de producción según '
                                        'combustible principal. La producción '
                                        'neta de las instalaciones no '
                                        'renovables e hidráulicas UGH tiene '
                                        'descontados los consumos propios de '
                                        'las instalaciones. En dichos tipos de '
                                        'producción una generación negativa '
                                        'indica que la electricidad horaria '
                                        'consumida para los usos de las '
                                        'plantas excede la producción horaria '
                                        'de las instalaciones.',
                         'last-update': '2019-06-12T17:40:40.000+02:00',
              

# OBTENCIÓN DE DATOS DE BALANCE




In [29]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [43]:
#def balance_datos(lang, category, widget, restaDia, input_año):

now = datetime.now()
ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()



lista_nombres = list()
lista_tipos = list()
lista_valores = list()
lista_porcentajes = list()
lista_fechas = list()
lista_dias = list()
lista_meses = list()
lista_años = list()


for dato in data['included']: 
    for info in dato['attributes']['content']:
        nombre = info['type']

        tipo = info['groupId']


        for i in info['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            dia = fecha.strftime('%d')
            mes = fecha.strftime('%m')
            año = fecha.strftime('%Y')

        
            
            
            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)
            lista_dias.append(dia)
            lista_meses.append(mes)
            lista_años.append(año)

df_balance = pd.DataFrame()
df_balance['nombre']               = lista_nombres
df_balance['tipo de energía']      = lista_tipos
df_balance['Valores']              = lista_valores
df_balance["Porcentaje"]           = lista_porcentajes
df_balance["Fecha actualización"]  = lista_fechas
df_balance["dia"]                  = lista_dias               
df_balance["mes"]                  = lista_meses               
df_balance["año"]                  = lista_años               




    #return df_balance  


In [45]:
lang    =  input('¿En español (es) o en inglés (en)?') 
category = input('¿Balance, demanda, generación o intercambio?') 
widget   = input('¿Qué widget quieres?')


restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

#balance_datos(lang, category, widget, restaDia, input_año)

In [32]:
#balance_datos(lang, category, widget, restaDia, input_año).to_csv('balance_electrico.csv')

In [46]:
df_balance

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización,dia,mes,año
0,Hidráulica,Renovable,25842.689,0.067826,2022-06-19 00:00:00+02:00,19,06,2022
1,Hidráulica,Renovable,34989.492,0.124016,2022-06-20 00:00:00+02:00,20,06,2022
2,Hidráulica,Renovable,38238.198,0.150150,2022-06-21 00:00:00+02:00,21,06,2022
3,Hidráulica,Renovable,31398.422,0.132560,2022-06-22 00:00:00+02:00,22,06,2022
4,Hidráulica,Renovable,23688.187,0.088852,2022-06-23 00:00:00+02:00,23,06,2022
...,...,...,...,...,...,...,...,...
2117,Demanda en b.c.,Demanda en b.c.,689988.523,1.000000,2022-09-23 00:00:00+02:00,23,09,2022
2118,Demanda en b.c.,Demanda en b.c.,602468.746,1.000000,2022-09-24 00:00:00+02:00,24,09,2022
2119,Demanda en b.c.,Demanda en b.c.,547492.934,1.000000,2022-09-25 00:00:00+02:00,25,09,2022
2120,Demanda en b.c.,Demanda en b.c.,638740.351,1.000000,2022-09-26 00:00:00+02:00,26,09,2022


In [47]:
df_balance.to_csv('balance_electrico.csv')